### Main Program

In [ ]:
## Julia program to read a selected .BVA file and display 30-minute time series plots
## JW October 2022
#using ContinuousWavelets 
using ContinuousWavelets, CSV
using Dates, DataFrames, Distributions, DSP
using FFTW
##using Gtk
using LaTeXStrings
using NativeFileDialog
using Plots
using Printf
using Statistics #, StatsPlots
#using Suppressor: @suppress
using Wavelets

##import Pkg; Pkg.add("Suppressor")
## See https://github.com/JuliaIO/Suppressor.jl
##using Suppressor: @suppress

include("./read_BVA_processing_tools.jl")
include("./read_BVA_plotting_tools.jl")

function handle_gaps(df)
################################################    
# a function to identify where gaps found in sequence numbers
#    where gaps occur a dummy record is inserted into the df
#    and both Status1 and Status2 are flagged with a '!' 
    
    nums = []

    # Convert sequence number value from Hex to Integer
    for i in 1:nrow(df)
        str = df.Sequence[i]
        push!(nums,parse(Int, str[1], base=16) * 16 + parse(Int, str[2], base=16))
    end

    # Determine number of gaps in df rows
    counter = diff(nums)
    gaps = findall(counter.<1)
    counter[gaps] .+= 256;

    # now find where a counter is > 1 indicating number of gaps in transmission
    gaps = findall(counter.>1)
    ll = length(gaps)
        
    if ll > 0
        
        println(cumsum(gaps)," found in file!")
        println("File contains ",nrow(df)," records")
    

##    df1 = deepcopy(df)

        # need to work through the gaps in reverse order in order to preserve row numbers
        for i in ll:-1:1
            # for each gap, get sequence number of last valid row
            sequence_number = df[gaps[i],:].Sequence

            # for number of gaps, insert "FFF" values into df and '!' into the status indicators
            for j in 1:counter[gaps[i]]-1

                sequence_number_hex = uppercase(string((parse(Int, sequence_number[1], base=16) * 16 + parse(Int, sequence_number[2], base=16)) + j, base=16, pad=2))
        ##        println(i,' ',j,' ',gaps[i]+j,' ',counter[gaps[i]],' ',sequence_number,' ',sequence_number_hex,' ',"!",' ',"FFFFFFFFFFFFFFFFFF",' ',"!",' ',"FFFFFF")
                insert!(df, gaps[i]+j, [sequence_number_hex, '!', "FFFFFFFFFFFFFFFFFF", '!', "FFFFFF"])

            end
                
        end

        println("File now contains ",nrow(df)," records")
                
    else
                
        println("No gaps in record")
                
    end

    return (df)
        
    end    # handle_gaps()
    

function string2hex(str)
################################################    
    parsed_val = (parse(Int, str[1], base=16) * 16 + parse(Int, str[2], base=16));
    if parsed_val == 0
        hex = 0x00
    else
        hex = [(parsed_val>>((i-1)<<3))%UInt8 for i in 1:sizeof(parsed_val)-leading_zeros(parsed_val)>>3][1]
    end
    
    return(hex)
    
    end    # string2hex()


################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Select a HVA or BVA file
##infil = pick_file("C:\\QGHL\\Wave_data\\Bris\\BVA\\", filterlist="HVA,BVA;hva,bva");
infil = pick_file("C:\\QGHL\\Wave_data\\Brisbane_offshore\\", filterlist="HVA,BVA;hva,bva");
println("Selected ",infil)
flush(stdout)

if uppercase(split(infil, ".")[end]) == "HVA"
    
#    df = DataFrame(CSV.File(infil,header=0, delim=",-"));
#    rename!(df,[:Sequence,:Data, :Packet]);
    
    # read data from file to df
    df = DataFrame(CSV.File(infil,header=0))

    # add column names to df as shown in DWTP Section 2.1 pp 18-19 - Status1; Data; Status2; Packet
    transform!(df, :Column2 => ByRow(x -> x[1]) => :Status1)
    transform!(df, :Column2 => ByRow(x -> x[2:19]) => :Data)
    transform!(df, :Column3 => ByRow(x -> x[1]) => :Status2)
    transform!(df, :Column3 => ByRow(x -> x[2:7]) => :Packet)

    rename!(df,:Column1 => :Sequence)
    DataFrames.select!(df, Not([:Column2, :Column3]))
            
    df1 = deepcopy(df)
    df = handle_gaps(df)    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    # identify rows where channel not received OK ( see DWTP 2.1 pp. 18-19)
    bad = findall(df[!,:Status1] .== '!')
    if !isempty(bad)
        println(length(bad)," unrecoverable records found!")
    else
        println("No unrecoverable records found!")
    end 

###    delete!(df, bad) # drop rows where channel is damaged beyond repair
    suspect = findall(df[!,:Status1] .== '=') # flag rows where channel is damaged but has been repaired successfully
    if !isempty(suspect)
        println(length(suspect)," damaged records found that have been repaired successfully!")
    else
        println("All records received OK!")
    end 

    # create a Packet vector of hex values
    packet = []
    
    for i in 1:length(df.Packet)
        push!(packet,string2hex(SubString(df.Packet[i],1,2)))
        push!(packet,string2hex(SubString(df.Packet[i],3,4)))
        push!(packet,string2hex(SubString(df.Packet[i],5,6)))
    end  
    
    # create a Data vector of hex values
    Data = lowercase.(df.Data)
    
elseif uppercase(split(infil, ".")[end]) == "BVA"
    
    #Change the type-interpretation of the binary file data to unsigned integer
    println("Reading BINARY data from ",infil)
    data = reinterpret(UInt8, read(infil));

    # turn the data vector into a matrix of 12 values matching hexadecimal bytes - see DWTP 2.1 p.18
    cols = 12
    rows = Int(length(data) / cols)
    mat = reshape(view(data, :), cols, :);

    # Interleave last 4 matrix columns to form packet vector
    ## based on mschauer @ https://discourse.julialang.org/t/combining-two-arrays-with-alternating-elements/15498/2
    packet = collect(Iterators.flatten(zip(mat[10,:],mat[11,:],mat[12,:])));
    
    ## get data for the Heave, North, and West displacements
    Data = []

    # Convert binary data to hexidecimal vectors
    j = 0
    println("Building displacements vectors - this takes a while!")
    while true

        try
            aa = []

            for i = j*12+1:j*12+12
                push!(aa,string(data[i], base = 16, pad = 2))
            end

            push!(Data,join(aa)[1:18])

        catch

            # escape if something is amiss        
            break

        end
        j = j+1

    end

else
    println("Not able to read this file type at present")
    exit()
end

# find all occurrences of 0x7e in packet vector
aa = findall(x->x.==0x7e, vec(packet));

# Create the df's to hold the processed data and setup their column structure
f20_vals = []; f21_vals = []; f23_vals = []; f25_vals = []; f26_vals = []; f28_vals = []; f29_vals = [];
    f80_vals = []; f81_vals = []; f82_vals = []; fc1_vals = []; fc3_vals = []

f20_df = DataFrame(Date = [], Segments = [], Smax = [])
for i in 0:99
    col_name = "S$i"
    f20_df[!,col_name] = []
end
f21_df = DataFrame(Date = [], Segments = [])

for i in 0:99
    col_name = "Dir$i"
    f21_df[!,col_name] = []
end
for i in 0:99
    col_name = "Spread$i"
    f21_df[!,col_name] = []
end
f23_df = DataFrame(Date = [], Segments = [], Match_vector = [], Sample_number = [])
f25_df = DataFrame(Date = [], Segments = [], Hs = [], Ti = [], Te = [], T1 = [], Tz = [], T3 = [], 
    Tc = [], Rp = [], Tp = [], Smax = [], Theta_p = [], σ_p = [])
f26_df = DataFrame(Date = [], Hmax = [], Thmax = [], Tmax = [], Htmax = [], Havg = [], Tavg = [], 
    Hsrms = [], Nw = [], Nc = [], Epsilon = [], Coverage = [])
f28_df = DataFrame(Date = [], Segments = [])
for i in 0:99
    col_name = "m2_$i"
    f28_df[!,col_name] = []
end
for i in 0:99
    col_name = "n2_$i"
    f28_df[!,col_name] = []
end
for i in 0:99
    col_name = "k$i"
    f28_df[!,col_name] = []
end
f29_df = DataFrame(Date = [], Coverage = [], Nw = [], Epsilon = [], Hmax = [], THmax = [], H10 = [], 
    TH10 = [], H3 = [], TH3 = [], Havg = [], Tavg = [])
for i in 0:22
    col_name = "Hq$i"
    f29_df[!,col_name] = []
end
f80_df = DataFrame(Date = [], Latitude = [], Longitude = [])
f81_df = DataFrame(Date = [], SST = [])
f82_df = DataFrame(Date = [], Firmware_Version = [], Speed = [], Direction = [], SST = [])
fc1_df = DataFrame(Date = [], Firmware = [], Hatch_uid = [], Hull_uid = [], Uptime = [], 
    Battery_energy = [], Boostcaps_energy = [],
    Hatch_temp = [], Battery_voltage = [], Batteries_per_section = [], Battery_section_number = [], 
    Initial_battery_energy = [], Ov = [], Cv = [], Ox = [], Oy = [], Cx = [], Cy = [], μ₀ = [], 
    σ₀ = [], μᵢ = [], σᵢ = [], μₕ = [], σh = [], Cpitch = [], Croll = [], Tensor = [])
fc3_df = DataFrame(Date = [], Battery_life = [])

# determine number of records
max_val = length(aa)-1

# Decode the packet data to messages
# refer to Section 2.1.2 Decoding the packet data to messages p. 20
messages = []

println("Processing the Packet vectors")
for i in 1:max_val
    # determine packet length
    first = aa[i]+1
    last = aa[i+1]
            
    list1 = []; list2 = []
    
    if (last-first > 1)
        global decoded = []
        decoded = packet[first:last-1]
        decode_length = length(decoded)
                
        bb = findall(x->x.==0x7d, vec(decoded));
            
        if bb != []

            # do an xor of elements with 0x7d
            for ii in bb
                if ii<(last-first)
                    decoded[ii+1] = decoded[ii+1] ⊻ 0x20 # set the xor value as 0x20 vide 2.1.2 p.20
                end
            end

            # remove the 0x7d
            deleteat!(decoded::Vector, bb)
            decode_length = length(decoded)

        end

##        println(string(decoded[2], base=16, pad=2),' ',length(decoded))

        # look for vectors of the spectrum synchronisation message (0xF23)
        if decoded[2] == 0x20
                    
            if decode_length !=161
                println("Alert 0xF20 message length is ",decode_length," but should be 161")
            else           
                heave_spectrum = []
                append!(f20_vals,decoded)
                timestamp,segments,smax,heave_spectrum = process_f20(decoded,heave_spectrum)         
                list_1 = [timestamp,segments,smax]
                push!(f20_df, [list_1; heave_spectrum])
            end
            
        elseif decoded[2] == 0x21
                    
            if decode_length !=309
                println("Alert 0xF21 message length is ",decode_length," but should be 309")
            else

                global direction = []
                global spread = []
                append!(f21_vals,decoded)
                timestamp,segments,direction,spread = process_f21(decoded,direction,spread)

                list1 = [timestamp,segments]
                list2 = [direction; spread]

                push!(f21_df, [list1; list2])

            end
                    
        elseif decoded[2] == 0x23
                  
            if decode_length !=22
                println("Alert 0xF23 message length is ",decode_length," but should be 22")
            else

                append!(f23_vals,decoded)
                timestamp,segments_used,match_vector,sample_number = process_f23(decoded)
                push!(f23_df, [timestamp,segments_used,match_vector,sample_number])

            end

        elseif decoded[2] == 0x25

            if decode_length !=27
                println("Alert 0xF25 message length is ",decode_length," but should be 27")
            else

                append!(f25_vals,decoded)
                timestamp,segments,hs,ti,te,t1,tz,t3,tc,rp,tp,smax,theta_p,σ_p = process_f25(decoded)
                push!(f25_df, [timestamp,segments,hs,ti,te,t1,tz,t3,tc,rp,tp,smax,theta_p,σ_p])
            end

            elseif decoded[2] == 0x26

            if decode_length !=25
                println("Alert 0xF26 message length is ",decode_length," but should be 25")
            else                    

                append!(f26_vals,decoded)
                timestamp,hmax,thmax,tmax,htmax,havg,tavg,hsrms,nw,nc,epsilon,coverage = process_f26(decoded)
                push!(f26_df, [timestamp,hmax,thmax,tmax,htmax,havg,tavg,hsrms,nw,nc,epsilon,coverage])
            end

        elseif decoded[2] == 0x28

            if decode_length !=459
                println("Alert 0xF28 message length is ",decode_length," but should be 459")
            else                    
            
                m2 = []
                n2 = []
                k = []
                append!(f28_vals,decoded)
                timestamp,segments,m2,n2,k = process_f28(decoded,m2,n2,k)

                global list1 = [timestamp,segments]
                global list2 = [m2; n2; k]

                push!(f28_df, [list1; list2])
            end
            
        elseif decoded[2] == 0x29

            if decode_length !=59
                println("Alert 0xF29 message length is ",decode_length," but should be 59")
            else                 
                    
                hq = []
                append!(f29_vals,decoded)
                timestamp,coverage,nw,epsilon,hmax,thmax,h10,th10,h3,th3,havg,tavg,hq = process_f29(decoded,hq)         
                list_1 = [timestamp,coverage,nw,epsilon,hmax,thmax,h10,th10,h3,th3,havg,tavg]
                push!(f29_df, [list_1; hq])
            end
        elseif decoded[2] == 0x80

            if decode_length !=14
                println("Alert 0xF80 message length is ",decode_length," but should be 14")
            else                 
            
                append!(f80_vals,decoded)
                timestamp,latitude,longitude = process_f80(decoded)
                push!(f80_df, [timestamp,latitude,longitude])
            end         
        elseif decoded[2] == 0x81

            if decode_length !=10
                println("Alert 0xF81 message length is ",decode_length," but should be 10")
            else                 
            
                append!(f81_vals,decoded)
                timestamp,sst = process_f81(decoded)
                push!(f81_df, [timestamp,sst])
            end
        elseif decoded[2] == 0x82

            if decode_length !=29
                println("Alert 0xF82 message length is ",decode_length," but should be 29")
            else                 
            
                append!(f82_vals,decoded)
                timestamp,firmware_version,speed,direction,sst = process_f82(decoded)
                push!(f82_df, [timestamp,firmware_version,speed,direction,sst])

            end
        elseif decoded[2] == 0xc1

            if decode_length !=67
                println("Alert 0xFC1 message length is ",decode_length," but should be 67")
            else                 
            
                append!(fc1_vals,decoded)
                timestamp,firmware,hatch_uid,hull_uid,uptime,battery_energy,boostcaps_energy,hatch_temp,battery_voltage,batteries_per_section,
                    battery_section_number,initial_battery_energy,ov,cv,ox,oy,cx,cy,μ₀,σ₀,μᵢ,σᵢ,μₕ,σh,cpitch,croll,tensor = process_fc1(decoded)   

                push!(fc1_df, [timestamp,firmware,hatch_uid,hull_uid,uptime,battery_energy,boostcaps_energy,hatch_temp,battery_voltage,batteries_per_section,
                    battery_section_number,initial_battery_energy,ov,cv,ox,oy,cx,cy,μ₀,σ₀,μᵢ,σᵢ,μₕ,σh,cpitch,croll,tensor])
            end
        elseif decoded[2] == 0xc3

            if decode_length !=9
                println("Alert 0xFC3 message length is ",decode_length," but should be 9")
            else                 
            
                append!(fc3_vals,decoded)
                timestamp,ble = process_fc3(decoded)
                push!(fc3_df, [timestamp,ble])

            end
        end
        
    end
    
end
    
# remove duplicates from dataframes
f20_df = unique(f20_df)
f21_df = unique(f21_df)
f23_df = unique(f23_df)
f25_df = unique(f25_df)
f26_df = unique(f26_df)    
f28_df = unique(f28_df)
f29_df = unique(f29_df)
f80_df = unique(f80_df)
f81_df = unique(f81_df)
f82_df = unique(f82_df)
fc1_df = unique(fc1_df)
fc3_df = unique(fc3_df)

println("All file data read!")
println("Preparing to plot data")
flush(stdout)
    
# remove those vectors from F23 df that are not located in the Data vector df
f23_df[!,"Data_vector"] = [findfirst(x->x==i, Data) for i in f23_df.Match_vector];

# Do time-series plot of available data
plot_f29(f29_df)

# Plot current speed and direction
plot_f82(f82_df)

println("Select date from menu for more plots")

################################################
################################################
##           END OF MAIN PROGRAM
################################################
################################################

In [ ]:
# Locate bad WSE data
bad_wse = findall(df.Status1.=='!')

# Locate bad Packet data
bad_packet = findall(df.Status2.=='!')

## Do Lagrangian plot of currents

In [ ]:
using Haversine
# see https://juliahub.com/ui/Packages/Haversine/1CvkY/1.1.0

# find closest Timestamp in 0xF82 to 0xF80 and use the λ and ϕ of this point as origin
start_point = findmin(abs.(f82_df.Date.-f80_df.Date[1]))[2]

latitudes = []; longitudes = []

p = GeoLocation(λ=f80_df.Longitude[start_point], ϕ= f80_df.Latitude[start_point])

push!(longitudes, p.λ)
push!(latitudes, p.ϕ)

for i in 2:nrow(f82_df)
    
    θ = f82_df.Direction[i-1]
    d = f82_df.Speed[i-1] * 600

    p = HaversineDestination(p, θ, d)
    
    push!(longitudes, p.λ)
    push!(latitudes, p.ϕ)
    
    p = GeoLocation(λ=p.λ, ϕ= p.ϕ)
    
end

f82_df.Lat = latitudes
f82_df.Long = longitudes;

# remove bad values
f82_df = filter(:Lat => x -> !any(f -> f(x), (ismissing, isnothing, isnan)), f82_df)

d2 = scatter(f82_df.Long, f82_df.Lat, marker=:circle, ms=:1, msw=:0, label="", xlabel="Longitude (°)", ylabel="Latitude (°)", 
    leftmargin = 20Plots.mm, bottommargin = 20Plots.mm,
    size=(800,800), aspect_ratio=:equal, framestyle = :box, title="Lagrangian current plot")
d2 = plot!(f82_df.Long, f82_df.Lat, lc=:lightblue, lw=:0.5, label="", fg_legend=:transparent, bg_legend=:transparent, foreground_color_grid="grey")
d2 = scatter!([first(f82_df.Long)],[first(f82_df.Lat)], marker=:utriangle, ms=:5, mc=:green, label="First "*Dates.format(first(f82_df.Date), "yyyy-mm-dd HH:MM:SS")*"\n")
d2 = scatter!([last(f82_df.Long)],[last(f82_df.Lat)], marker=:dtriangle, ms=:5, mc=:red, label="Last "*Dates.format(last(f82_df.Date), "yyyy-mm-dd HH:MM:SS"))

ps = plot(d2)

display(ps)

# Select individual records to plot

## Select individual wave records

In [ ]:
function plot_scaleogram(heave, start_date)
################################################
    n=length(heave);
    t = range(1,n/2.56,length=n);
    
    # time stamp each WSE
    points = collect(0:1:length(heave)-1)/2.56
    times = []

    for i in 1:length(points)
        push!(times,unix2datetime(datetime2unix(start_date) + points[i]))
    end

    c = wavelet(Morlet(8), β=0.75);
    res = ContinuousWavelets.cwt(heave, c)

    freqs = getMeanFreq(ContinuousWavelets.computeWavelets(n, c)[1])
#    freqs[1] = 0

    # display plots to screen
    tm_tick = range(first(times),last(times),step=Minute(5))
    ticks = Dates.format.(tm_tick,"HH:MM:SS")

    p1 = heatmap(times, ((freqs.-minimum(freqs))./maximum(freqs)).*0.64, abs.(res)', c=cgrad(:Spectral, rev=true))                

    for i in 0:0.1:1.28
        hline!(p1, [i], lw=0.5, c=:white, label="")
    end

    start_date = first(times)
    last_date = last(times)

    for i in start_date:Minute(5):last_date
        vline!(p1, [i], lw=0.5, c=:white, label="")
    end

    # Plot spectrogram over scalogram
    nw=128;
    spec = DSP.Periodograms.spectrogram(heave, nw, 120; fs=2.56,window=hanning);

    # display plots to screen
    tm_tick = range(first(times),last(times),step=Minute(5))
    ticks = Dates.format.(tm_tick,"HH:MM:SS")

    p1 = plot!(first(times) + Microsecond.(ceil.((spec.time) * 1000000)), spec.freq, DSP.Periodograms.power(spec), lw=1, c=cgrad(:Spectral, rev=true), colorbar=false) 

    plot_wavelet = plot(p1, 
        xlabel="Time", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
        title="Scaleogram " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1500, 500), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    # display plots to screen
    display(plot_wavelet)
    
    return()
    
    end    # plot_scaleogram()

    
## Plot 30-minute records
# create a vector of dates from the F23 df
vector = Dates.format.(f23_df.Date, "yyyy-mm-dd HH:MM:SS");

## Allow user to get a 30-minute record and do plots
cb = GtkComboBoxText()
choices = vector

for choice in choices
    push!(cb,choice)
end

set_gtk_property!(cb,:active,1)

signal_connect(cb, "changed") do widget, others...

    # get the active index
    idx = get_gtk_property(cb, "active", Int) + 2
    println(idx)
  
    # get the active string 
    str = Gtk.bytestring( GAccessor.active_text(cb) ) 
    
    plot_hnw(f23_df,fc1_df,Data,idx)
    plot_spectra(f23_df,f20_df,Data,idx)
##    plot_2d(f23_df,Data,idx)
##    plot_hnw_2d(f23_df,Data,idx)
##    plot_3d(f23_df,Data,idx)
    plot_heave_histogram(f23_df,Data,idx)
#    plot_wavelet(f23_df,Data,idx)
        
    global start_date, start_val, end_val = get_start_end_dates(f23_df,idx)

    # get WSEs for desired 30-minute record
    global heave, north, west = get_hnw(Data,start_val,end_val)
    plot_scaleogram(heave, start_date)

end

win = GtkWindow("Select Date",200,200);
Gtk.GAccessor.position(win, Gtk.GtkWindowPosition.CENTER);
push!(win, cb);
showall(win);

In [ ]:
using Plots
using CurveFit

y = heave[2600:3100]
x = 1:1:length(y)

px = plot(x, y, label="")

fit = curve_fit(Polynomial, x, y, 30)
yb = fit.(x) 

p1 = plot!(x, yb, label="")

px_plot = plot(px, size=(1800,600))

display(px_plot)

In [ ]:
plot(x, y, marker=:o, ms=:2, label="", size=(1800,600))

fit = curve_fit(Polynomial, x, y, 55)
yb = y - fit.(x) 

plot!(x, yb, label="")

### Investigate LombScargle package

In [ ]:
using Plots
using LombScargle

Sample_frequency = 2.56

N = length(heave)
t = collect(1:N)/Sample_frequency

plan = LombScargle.plan(t, heave, normalization=:psd, fit_mean=:false, center_data=:false, samples_per_peak=:20, maximum_frequency=:0.64);
pgram = lombscargle(plan)

p = periodogram(heave; onesided=true, nfft=N, fs=2.56)

ps_w = welch_pgram(heave, 512, 256; onesided=true, nfft=512, fs=Sample_frequency, window=hanning);
f2 = freq(ps_w);
Pden2 = power(ps_w);

p1 = plot(freqpower(pgram)...,xrange=(0.0,0.64), c=:yellow, lw=:3, label="Lomb Scargle\n")
p1 = plot!(freq(p), power(p), label="DSP Periodogram\n", c=:blue, lw=:0.5)
p1 = plot!(f2,Pden2, label="DSP Welch's method", c=:red, lw=:1)
p2 = plot(p1, size=(1200,800), framestyle = :box, fg_legend=:transparent)

display(p2)


### Investigate Skewness and Kurtosis calculations

In [ ]:
skew = skewness(heave)
kurt = kurtosis(heave)

#extrema(heave)

lim=round(maximum(abs.(extrema(heave))) + 0.2; digits = 1)

fit_distribution = fit_mle(Normal,heave)
bin_vals = vcat(reverse(collect(-0.2:-0.2:-lim)),collect(0:0.2:lim))
        
h = histogram(heave, normalize=:false, bins=bin_vals, alpha=0.5)
bin_count = fit(Histogram, heave, nbins=length(bins)).weights
h = plot!(twinx(), [-lim:0.01:lim],[pdf(fit_distribution,i) for i in -lim:0.01:lim], lc=:blue, lw=:1, ls=:dash)

show_normal_plot = plot(h,
        framestyle = :box, size = (1400, 800), legend=:topleft, xlim=(first(bin_vals),last(bin_vals)),
        leftmargin = 20Plots.mm, rightmargin = 20Plots.mm, fg_legend=:transparent)

display(show_normal_plot)

bins = collect(collect(fit(Histogram, heave, nbins=length(bins)).edges)[1])
println("    Bins       Count")
for i in 1:1:length(bin_count)
    @printf("%4.1f to %4.1f %7d\n",bins[i],bins[i+1],bin_count[i])
end

@printf("Skewness = %4.4f\n",skew)
@printf("Kurtosis = %4.4f\n",kurt)

## Show individual WSE's and zero-crossing points

In [ ]:
found_list = 2 # <<<=== For testing only

start_date, start_val, end_val = get_start_end_dates(f23_df,found_list)
    
# get WSEs for desired 30-minute record
heave, north, west = get_hnw(Data,start_val,end_val);

In [ ]:
zero_up = []; valid_zero_up = []

for i in 2:length(heave)-1
    if (heave[i]*heave[i+1] < 0 && heave[i+1] > 0) || (heave[i] == 0 && heave[i-1] < 0 && heave[i+1] > 0)
        push!(zero_up,i)
    end
end

##med = median(heave)
##stdev = std(heave)
##stdev_3 = med + stdev*3

wse_point = 1:1:length(heave)
t = wse_point./2.56

wse_1 = plot(t, heave[wse_point], lw=2, c=:blue, alpha=.5, label = "WSE's", fillalpha = 0.0075, fillcolor = :blue)
wse_1 = scatter!(t, heave[wse_point], c=:white, ms=3, 
    markerstrokecolor=:blue, alpha=0.5, markerstrokewidth=0.5, label="WSE points")
wse_1 = scatter!(zero_up[1:24]./2.56, heave[zero_up[1:24]], ms=4, c=:lightgreen, xlim=(1,20), 
    markerstrokecolor=:green, alpha=0.5, series_annotations = text.(zero_up, :bottom, :red, :size, 10), 
    annotationhalign = :hcenter, label="Zero up-cross points")

# heave Threshold set at 10mm. Refer to Section 9 Wave statistics pp. 9-10 in Datawell Library Manual
threshold = 0.05

wse_1 = hline!([threshold; threshold], lw=0.2, ls =:dot, c=:red, label="Threshold ("*L"\pm"*string(threshold)*")\n")
wse_1 = hline!([-threshold; -threshold], lw=0.2, ls =:dot, c=:red, label="")

##wse_1 = hline!([stdev_3; stdev_3], lw=0.2, ls =:dot, c=:green, label="3 sigma")
##wse_1 = hline!([-stdev_3; -stdev_3], lw=0.2, ls =:dot,  c=:green, label="")

##wse_plot = plot(wse_point,wse_1, size = (1400, 600),xlim=(length(heave)-100,length(heave)), ylim=(-1.5,1.5), framestyle = :box, 
wse_plot = plot(wse_1, size = (1400, 600),xlim=(0,100), xlabel="Time (s)", ylabel="Displacement (m)", ylim=(-1.5,1.5), framestyle = :box,     
    fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    margin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true)

#save plot
savefig(".\\plot.png")

display(wse_plot)

## Identify individual waves and calculate time-domain heights

In [ ]:
valid_zero_up = []; crest_points = []; trough_points = []
i = 1; j = 2

while j < length(zero_up)-1
    
    crest = maximum(heave[zero_up[i]:zero_up[j]])
    crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
    trough = minimum(heave[crest_point:zero_up[j]])

    # Check that crest higher than threshold AND trough less than threshold - Possible Valid Wave!!
    if (crest > threshold) & (trough < -threshold)
        crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
        trough_point = crest_point + argmin(heave[crest_point:zero_up[j]]) - 1
        
        push!(crest_points,crest_point)
        push!(trough_points,trough_point)
        
        next_crest = maximum(heave[zero_up[j]:zero_up[j+1]])
        
        # Check that NEXT crest also exceeds threshold (if so then Valid Wave)
        if (next_crest > threshold)
##            println("Crest found at ",crest_point," Trough at ",trough_point)
            push!(valid_zero_up,(zero_up[i],zero_up[j]));
            i = j
        end
        
    end

    j = j+1
    
end

# Process last recorded wave
#i = j
#j = j+1

crest = maximum(heave[zero_up[i]:zero_up[j]])
trough = minimum(heave[zero_up[i]:zero_up[j]])

if (crest > threshold) & (trough < -threshold)

    crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
    trough_point = crest_point + argmin(heave[crest_point:zero_up[j]]) - 1
    push!(valid_zero_up,(zero_up[i],zero_up[j]));

end

heights = []

for i in 1:length(valid_zero_up)
    
    crest = maximum(heave[valid_zero_up[i][1]:valid_zero_up[i][2]]);
    trough = minimum(heave[valid_zero_up[i][1]:valid_zero_up[i][2]]);
    push!(heights,crest - trough)
##    @printf("Wave %d = %2.3f\n",i,crest - trough)

end 

# Get time-domain height parameters
sorted_heights = sort(heights, rev=true) # sort heights in reverse order heighestwave to loheave wave
hmax = maximum(sorted_heights)
hs = mean(sorted_heights[1:Int(ceil(length(sorted_heights)/3))])
h10 = mean(sorted_heights[1:Int(ceil(length(sorted_heights) / 10))])
hmean = mean(sorted_heights)

@printf("%s; Waves = %3d; Hmean = %4.2fm; Hs = %4.2fm; H10 = %4.2fm; Hmax = %4.2fm\n",Dates.format(start_date, "yyyy-mm-dd HH:MM"),length(heights), hmean, hs, h10, hmax)

## Locate the zero-crossing points

x_point = []
for i in 1:length(valid_zero_up)
    push!(x_point,valid_zero_up[i][1] + abs(heave[valid_zero_up[i][1]]) / (heave[valid_zero_up[i][1]+1] - heave[valid_zero_up[i][1]]))
end

# Process final zero-crossing point
i = length(valid_zero_up)
push!(x_point,valid_zero_up[i][2] + abs(heave[valid_zero_up[i][2]]) / (heave[valid_zero_up[i][2]+1] - heave[valid_zero_up[i][2]]))

ix = 15

# Do plots
wse_1 = plot(wse_point./2.56, heave[wse_point], lw=2, c=:blue, alpha=0.5, label = "WSE's", fillrange = 0, fillalpha = 0.0075, fillcolor = :blue)
wse_1 = scatter!(wse_point./2.56, heave[wse_point], c=:white, ms=3, 
    markerstrokecolor=:blue, alpha=0.5, markerstrokewidth=0.5,label="WSE points")
wse_1 = scatter!(zero_up[1:ix]./2.56, heave[zero_up[1:ix]], ms=3, c=:lightgreen, 
    markerstrokecolor=:lightgreen, series_annotations = text.(zero_up, :bottom, :red, :size, 10), 
    annotationhalign = :hcenter, label="Zero up-cross points\n")

# heave Threshold set at 10mm. Refer to Section 9 Wave statistics pp. 9-10 in Datawell Library Manual
threshold = 0.05 

wse_1 = hline!([threshold; threshold], lw=0.2, ls =:dot, c=:red, label="Threshold\n")
wse_1 = hline!([-threshold; -threshold], lw=0.2, ls =:dot, c=:red, label="")

wse_1 = scatter!(x_point./2.56, zeros(length(x_point)), c=:yellow, ms=5, 
    markerstrokecolor=:yellow, markershape=:diamond, label="Zero-crossing points")


#wse_plot = plot(wse_1, size = (1400, 800),xlim=(length(heave)-100,length(heave)), ylim=(-1.5,1.5), framestyle = :box, 
wse_plot = plot(wse_1, size = (1400, 600), xlabel="Time (s)", ylabel="Displacement (m)", xlim=(20,60), ylim=(-1.5,1.5), framestyle = :box,     
    fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    margin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true)

#save plot
savefig(".\\plot.png")

display(wse_plot)

## Calculate time-domain periods

In [ ]:
params_df = DataFrame(Date = [], Hmax = [], Thmax = [], Tmax = [], Htmax = [], Havg = [], 
    Tavg = [], Hsrms = [], Nw = [], Nc = [], Epsilon = [], Coverage = [])
sample_frequency = 2.56 # Hertz Mk4
periods = []

for i in 1:length(x_point)-1
    push!(periods,(x_point[i+1]-x_point[i]) / sample_frequency) # wave period in seconds
end

sorted_periods = sort(periods, rev=true) # sort periods in reverse order longest period to shortest period
tmean = mean(sorted_periods)
ths = periods[argmin(abs.(heights .- hs))] 
th10 = periods[argmin(abs.(heights .- h10))]
thmax = periods[argmax(heights)]
tmax = maximum(sorted_periods)

# get Datawell parameters from f29_df
row = f29_df[f29_df.Date .== start_date + Minute(30), :]

# Print results
@printf("\nQGHL values:     ")
@printf("%s; Waves = %3d; Hmean = %5.2fm; Hs = %5.2fm; H10 = %5.2fm; Hmax = %5.2fm;",Dates.format(start_date, "yyyy-mm-dd HH:MM"),length(heights), hmean, hs, h10, hmax)
@printf(" Tmean = %5.2fs; THs = %5.2fs; TH10 = %5.2fs; THmax = %5.2fs; Tmax = %5.2fs",tmean,ths,th10,thmax,tmax)

@printf("\nDatawell values: ")
@printf("%s; Waves = %3d; Hmean = %5.2fm; Hs = %5.2fm; H10 = %5.2fm; Hmax = %5.2fm;",Dates.format(row.Date[1], "yyyy-mm-dd HH:MM"),row.Nw[1], row.Havg[1], row.H3[1], row.H10[1], row.Hmax[1])
@printf(" Tmean = %5.2fs; THs = %5.2fs; TH10 = %5.2fs; THmax = %5.2fs\n",row.Tavg[1],row.TH3[1],row.TH10[1],row.THmax[1])


## Plot wave heights and periods

title_string = Dates.format(start_date, "dd/mm/yyyy HH:MM")

wave_heights = hline([hmax; hmax], lw=1, c=:red, label="")
wave_heights = hline!([h10; h10], lw=1.5, c=:pink, label="")
wave_heights = hline!([hs; hs], lw=1, c=:blue, label="")
wave_heights = hline!([hmean; hmean], lw=1.5, c=:green, label="")

wave_heights = annotate!([15], [hmax*1.05], text("Hmax " * string(round(hmax, digits=2)) * "m", :red, 10), :top)
wave_heights = annotate!([15], [h10*1.05], text("H10 " * string(round(h10, digits=2)) * "m", :pink, 10), :top)
wave_heights = annotate!([15], [hs*1.05], text("Hs " * string(round(hs, digits=2)) * "m", :blue, 10), :top)
wave_heights = annotate!([15], [hmean*1.05], text("Hmean " * string(round(hmean, digits=2)) * "m", :green, 10), :top)


wave_heights = plot!(heights, lw=2, c=:"#4a536b", fillrange = 0, fillalpha = 0.035, fillcolor = :"#4a536b",
    ylim=(0,hmax*1.1), title=title_string, ylabel="Wave height (m)", label="")

min_h3_wave = minimum(sorted_heights[1:Int(ceil(length(sorted_heights)/3))])
# show which waves were used in calculation of Hs
wave_heights = scatter!(findall(i->(i>=min_h3_wave), heights),heights[findall(i->(i>=min_h3_wave), heights)], ms=4, mc=:lightblue, ma=0.25, label="")

wave_periods = hline([tmax; tmax], lw=1, c=:red, label="")
wave_periods = hline!([thmax; thmax], lw=1.5, c=:pink, label="")
wave_periods = hline!([ths; ths], lw=1, c=:blue, label="")

wave_periods = annotate!([15], [tmax*1.05], text("Tmax " * string(round(tmax, digits=2)) * "s", :red, 10), :top)
wave_periods = annotate!([15], [thmax*1.05], text("THmax " * string(round(thmax, digits=2)) * "s", :pink, 10), :top)
wave_periods = annotate!([15], [ths*1.05], text("THs " * string(round(ths, digits=2)) * "s", :blue, 10), :top)

wave_periods = plot!(periods, lw=2, c=:red, fillrange = 0, fillalpha = 0.025, fillcolor = :red, label="",
    ylim=(0,tmax*1.1), xlabel="Wave number", ylabel="Wave period (s)")

wave_heights_plot = plot(wave_heights, wave_periods, layout = (2, 1), size = (1400, 800), xlim=(0,length(heights)*1.015),
    framestyle = :box, titlefontsize=12, fg_legend=:transparent, bg_legend=:transparent,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true)

display(wave_heights_plot)

## Plot normal distribution on WSEs

In [ ]:
global title_string = "JJ"
plot_normal(heights)

## Do Rayleigh plot of wave heights

In [ ]:
plot_rayleigh(heights)

## Calculate declination and Inclination for record

In [ ]:
##############################################################################################
##############################################################################################
## Using model located at http://www.geomag.bgs.ac.uk/data_service/models_compass/wmm_calc.html
## But different values from https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfgrid
##############################################################################################
##############################################################################################

using XLSX
using CategoricalArrays

function get_lower_and_upper(year_dec_df, year_inc_df, lower_lat, lower_lon, upper_lat, upper_lon)
################################################    

    # get the declinations for lower latitude
    lower_lat_lower_lon_declination = year_dec_df[findfirst(year_dec_df.Lat .== lower_lat),string(lower_lon)]
##    println("Lower Lat & Lower Lon ", lower_lat,' ',lower_lon,' ' ,lower_lat_lower_lon_declination)
    lower_lat_upper_lon_declination = year_dec_df[findfirst(year_dec_df.Lat .== lower_lat),string(upper_lon)]
##    println("Lower Lat & Upper Lon ", lower_lat,' ',upper_lon,' ' ,lower_lat_upper_lon_declination)

    # get the declinations for upper latitude
    upper_lat_lower_lon_declination = year_dec_df[findfirst(year_dec_df.Lat .== upper_lat),string(lower_lon)]
##    println("Upper Lat & Lower Lon ", upper_lat,' ',lower_lon,' ' ,upper_lat_lower_lon_declination)
    upper_lat_upper_lon_declination = year_dec_df[findfirst(year_dec_df.Lat .== upper_lat),string(upper_lon)]
##    println("Upper Lat & Upper Lon ", upper_lat,' ',upper_lon,' ' ,upper_lat_upper_lon_declination)

    # get the inclinations for lower latitude
    lower_lat_lower_lon_inclination = year_inc_df[findfirst(year_inc_df.Lat .== lower_lat),string(lower_lon)]
##    println("Lower Lat & Lower Lon ", lower_lat,' ',lower_lon,' ' ,lower_lat_lower_lon_inclination)
    lower_lat_upper_lon_inclination = year_inc_df[findfirst(year_inc_df.Lat .== lower_lat),string(upper_lon)]
##    println("Lower Lat & Upper Lon ", lower_lat,' ',upper_lon,' ' ,lower_lat_upper_lon_inclination)

    # get the inclinations for upper latitude
    upper_lat_lower_lon_inclination = year_inc_df[findfirst(year_inc_df.Lat .== upper_lat),string(lower_lon)]
##    println("Upper Lat & Lower Lon ", upper_lat,' ',lower_lon,' ' ,upper_lat_lower_lon_inclination)
    upper_lat_upper_lon_inclination = year_inc_df[findfirst(year_inc_df.Lat .== upper_lat),string(upper_lon)]
##    println("Upper Lat & Upper Lon ", upper_lat,' ',upper_lon,' ' ,upper_lat_upper_lon_inclination)
    
    return(lower_lat_lower_lon_declination, lower_lat_upper_lon_declination, upper_lat_upper_lon_declination, upper_lat_lower_lon_declination, 
                lower_lat_lower_lon_inclination, lower_lat_upper_lon_inclination, upper_lat_upper_lon_inclination, upper_lat_lower_lon_inclination)
    
end    # get_lower_and_upper()
    

function get_point_value(lat_diff, lon_diff, upper_lower, lower_lower, upper_upper, lower_upper)
################################################    
# function to calculate declination or inclination of point located with grid cell
    
    # first using latitude differences
    lower_diff = upper_lower - lower_lower
    upper_diff = upper_upper - lower_upper

##    println(lower_diff,' ',upper_diff)

    lower_record = lower_lower - lower_diff*lat_diff
    upper_record = lower_upper - upper_diff*lat_diff

    lon_record_using_latitude = lower_record + (upper_record-lower_record)*lon_diff

##    println(lower_record,' ',upper_record,' ',lon_record_using_latitude)
    
    # second using longitude differences
    lower_diff = lower_upper - lower_lower
    upper_diff = upper_upper - upper_lower

##    println(lower_diff,' ',upper_diff)

    lower_record = lower_lower + lower_diff*lon_diff
    upper_record = upper_lower + upper_diff*lon_diff

    lon_record_using_longitude = lower_record - (upper_record-lower_record)*lat_diff

##    println(lower_record,' ',upper_record,' ',lon_record_using_longitude)
    
    # return mean value of the latitude and longitude calculations
    return(mean([lon_record_using_longitude,lon_record_using_latitude]))
    
end    # get_point_value()


# Get approximate date of record
start_year = Dates.year(f80_df.Date[1])
end_year = start_year + 1
record_month = Dates.month(f80_df.Date[1])

@printf("Calculation date = 01/%2.2d/%4.4d using World Magnetic Model\n", record_month, start_year)
# Determine first worksheet number
start_sheet = indexin(start_year,[2023, 2024, 2025])[1] + 1
end_sheet = start_sheet + 1

# Read Declination and Inclination values from file
excel_directory = ".\\" 
excel_file = excel_directory*"Declination_and_Inclination_2023_2024.xlsx"
buoys_df = DataFrame(XLSX.readtable(excel_file,1,))
start_dec_df = DataFrame(XLSX.readtable(excel_file,start_sheet,))
end_dec_df = DataFrame(XLSX.readtable(excel_file,end_sheet,))
                
start_inc_df = DataFrame(XLSX.readtable(excel_file,start_sheet+3,))
end_inc_df = DataFrame(XLSX.readtable(excel_file,end_sheet+3,));

# Get a representative Lat and Lon for the record
record_lat = (mode(f80_df.Latitude))
record_lon = mode(f80_df.Longitude);
@printf("Buoy position from GPS: %4.5f° %4.5f°\n", record_lat, record_lon)

#get integer latitudes above and below record latitude
lower_lat = trunc(Int, record_lat)
upper_lat = lower_lat - 1

#get integer longitudes above and below record longitude
lower_lon = trunc(Int, record_lon)
upper_lon = lower_lon + 1


start_lower_lat_lower_lon_declination, start_lower_lat_upper_lon_declination, start_upper_lat_upper_lon_declination, start_upper_lat_lower_lon_declination, 
    start_lower_lat_lower_lon_inclination, start_lower_lat_upper_lon_inclination, start_upper_lat_upper_lon_inclination, start_upper_lat_lower_lon_inclination =
        get_lower_and_upper(start_dec_df, start_inc_df, lower_lat, lower_lon, upper_lat, upper_lon)

end_lower_lat_lower_lon_declination, end_lower_lat_upper_lon_declination, end_upper_lat_upper_lon_declination, end_upper_lat_lower_lon_declination, 
    end_lower_lat_lower_lon_inclination, end_lower_lat_upper_lon_inclination, end_upper_lat_upper_lon_inclination, end_upper_lat_lower_lon_inclination =
        get_lower_and_upper(end_dec_df, end_inc_df, lower_lat, lower_lon, upper_lat, upper_lon)

lat_diff = record_lat - lower_lat
lon_diff = record_lon - lower_lon

# calculate declination and inclination of records position at start of year
start_declination = get_point_value(lat_diff, lon_diff, start_upper_lat_lower_lon_declination, start_lower_lat_lower_lon_declination, start_upper_lat_upper_lon_declination, start_lower_lat_upper_lon_declination)
start_inclination = get_point_value(lat_diff, lon_diff, start_upper_lat_lower_lon_inclination, start_lower_lat_lower_lon_inclination, start_upper_lat_upper_lon_inclination, start_lower_lat_upper_lon_inclination)

# calculate declination and inclination of records position at end of year
end_declination = get_point_value(lat_diff, lon_diff, end_upper_lat_lower_lon_declination, end_lower_lat_lower_lon_declination, end_upper_lat_upper_lon_declination, end_lower_lat_upper_lon_declination)
end_inclination = get_point_value(lat_diff, lon_diff, end_upper_lat_lower_lon_inclination, end_lower_lat_lower_lon_inclination, end_upper_lat_upper_lon_inclination, end_lower_lat_upper_lon_inclination)

@printf("01/01/%4d Declination = %4.5f; Inclination = %4.5f\n", start_year, start_declination, start_inclination)
@printf("01/01/%4d Declination = %4.5f; Inclination = %4.5f\n", end_year, end_declination, end_inclination)

# get the declination and inclination using start date + month / 12
declination = start_declination + (end_declination-start_declination)*record_month/12
inclination = start_inclination + (end_inclination-start_inclination)*record_month/12
@printf("01/%2.2d/%4d Declination = %4.5f; Inclination = %4.5f\n", record_month, start_year, declination, inclination)

In [ ]:
## Compute the estimated values of magnetic declination and inclination based on the current World Magnetic Model (WMM) or the International Geomagnetic Reference Field (IGRF) model.
using XLSX
using CategoricalArrays

# Read Declination and Inclination values from file
excel_directory = ".\\" 

# Select 
##excel_file = excel_directory*"IGRF_13_Declination_and_Inclination.xlsx"
excel_file = excel_directory*"WMM_2020_2024_Declination_and_Inclination.xlsx"

dec_df = DataFrame(XLSX.readtable(excel_file,1,))
inc_df = DataFrame(XLSX.readtable(excel_file,2,));


In [ ]:
# Get the declination and inclination of the four points bounding the record position
dec_box = dec_df[(dec_df.Year .== start_year) .&& (upper_lat .<= dec_df.Latitude .<= lower_lat) .&& (lower_lon .<= dec_df.Longitude .<= upper_lon),:]
inc_box = inc_df[(inc_df.Year .== start_year) .&& (upper_lat .<= inc_df.Latitude .<= lower_lat) .&& (lower_lon .<= inc_df.Longitude .<= upper_lon),:]

In [ ]:
# Get the declination and inclination of the four points bounding the record position
dec_box = dec_df[(dec_df.Year .== start_year) .&& (upper_lat .<= dec_df.Latitude .<= lower_lat) .&& (lower_lon .<= dec_df.Longitude .<= upper_lon),:]
inc_box = inc_df[(inc_df.Year .== start_year) .&& (upper_lat .<= inc_df.Latitude .<= lower_lat) .&& (lower_lon .<= inc_df.Longitude .<= upper_lon),:]

dec_vals = dec_box.Declination
inc_vals = inc_box.Inclination

In [ ]:
dec_box

In [ ]:
dec_box = dec_df[(dec_df.Year .== start_year) .&& (upper_lat .<= dec_df.Latitude .<= lower_lat) .&& (lower_lon .<= dec_df.Longitude .<= upper_lon),:]
inc_box = inc_df[(inc_df.Year .== start_year) .&& (upper_lat .<= inc_df.Latitude .<= lower_lat) .&& (lower_lon .<= inc_df.Longitude .<= upper_lon),:]
dec_box

In [ ]:
get_point_value(lat_diff, lon_diff, start_upper_lat_lower_lon_declination, start_lower_lat_lower_lon_declination, start_upper_lat_upper_lon_declination, start_lower_lat_upper_lon_declination)

In [ ]:
aa = dec_box.Declination
bb = inc_box.Inclination

get_point_value(lat_diff, lon_diff, aa[2], aa[1], aa[4], aa[3])

In [ ]:
stat(infil)

In [ ]:
3428383 /48

In [ ]:
readdir()

In [ ]:
infil = "C:\\QGHL\\Wave_data\\Bilinga\\Bilinga_4224}2021-09-04T00h00Z.hva"

In [ ]:
splitpath(splitext(infil)[1])[end]

In [ ]:
splitext(splitpath(infil)[end])[1]

In [ ]:
content = open(infil) do f
  seekend(f)
  skip(f, -1)
  aa = read(f, String);
end

In [ ]:
f23_df[13:33,:]

In [ ]:
displacements[findall(>=(2048), displacements)] = displacements[findall(>=(2048), displacements)] .- 4096;

In [ ]:
plot(0.457*sinh.(displacements/457), size=(1800,500), xlim=(50000,75000))

In [ ]:
displacements = []
    
for i in arry
    append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
end

In [ ]:
plot(heave)

In [ ]:
4608*5